##### Copyright 2018 The TensorFlow Authors. [Licensed under the Apache License, Version 2.0](#scrollTo=Afd8bu4xJOgh).

In [ ]:
// #@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
// https://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/swift/tutorials/custom_differentiation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/custom_differentiation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/swift/blob/master/docs/site/tutorials/custom_differentiation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
</table>

# カスタム微分

このチュートリアルでは、わずか 5 行の Swift によって独自のカスタム導関数を定義し、導関数の手直しを行い、独自の勾配チェックポイント API を実装する方法を紹介します。

## カスタム導関数を宣言する

微分可能なパラメータと結果を持つ、任意の Swift 関数のカスタム導関数を定義することができます。これにより、C 関数をインポートして微分可能にすることもできます。

In [ ]:
import Glibc

func sillyExp(_ x: Float) -> Float {
    let 𝑒 = Float(M_E)
    print("Taking 𝑒(\(𝑒)) to the power of \(x)!")
    return pow(𝑒, x)
}

@derivative(of: sillyExp)
func sillyDerivative(_ x: Float) -> (value: Float, pullback: (Float) -> Float) {
    let y = sillyExp(x)
    return (value: y, pullback: { v in v * y })
}

print("exp(3) =", sillyExp(3))
print("𝛁exp(3) =", gradient(of: sillyExp)(3))

## 導関数の伝播を防ぐ

機械学習のユースケースで「勾配を止める」として一般的に知られている `withoutDerivative(at:)` メソッドは、導関数の伝播を防ぎます。

さらに、`withoutDerivative(at:)` は微分してはいけないものを識別し、より効率的な導関数を生成するため、 Swift コンパイラの役に立つ場合があります。関数の微分が常にゼロになることを検出可能な場合は Swift コンパイラが警告を生成します。明示的に `withoutDerivative(at:)` を使用すると、その警告は非表示になります。

In [ ]:
let x: Float = 2.0
let y: Float = 3.0
gradient(at: x, y) { x, y in
    sin(sin(sin(x))) + withoutDerivative(at: cos(cos(cos(y))))
}

## 導関数の手直し

[`withDerivative(_:)`](https://www.tensorflow.org/swift/api_docs/Protocols/Differentiable#/s:10TensorFlow14DifferentiablePAAE12withGradientyxy15CotangentVectorQzzcF) メソッドは、囲む関数のバックプロパゲーション中の値で、任意の演算（突然変異を含む）を勾配上で実行させます。

これを使用して、デバッグを行ったり実験的にバックプロパゲーションを調整したりすることができます。

### どこでも動作する

標準ライブラリが提供しているすべての微分 API は、 `Differentiable` プロトコルに準拠するすべての型を一般的に定義します。これには `Float`、`Double`、`Float80`、SIMD ベクトル、そして独自の型も含まれます！

`Differentiable` プロトコルの詳細については、テクニカルドキュメントの[可微分の型](https://github.com/tensorflow/swift/blob/master/docs/DifferentiableTypes.md)をご覧ください。

In [ ]:
var x: Float = 30
gradient(at: x) { x -> Float in
    // Print the partial derivative with respect to the result of `sin(x)`.
    let a = sin(x).withDerivative { print("∂+/∂sin = \($0)") } 
    // Force the partial derivative with respect to `x` to be `0.5`.
    let b = log(x.withDerivative { (dx: inout Float) in
        print("∂log/∂x = \(dx), but rewritten to 0.5");
        dx = 0.5
    })
    return a + b
}

### ニューラルネットワークモジュール内で使用する

単純な `Float` 関数で使うのと同じように、これはあらゆる数値的アプリケーションで使用することができます。[Swift for TensorFlow ディープラーニングライブラリ](https://github.com/tensorflow/swift-apis)を使用して構築された以下のニューラルネットワークもその 1 つです。

In [ ]:
import TensorFlow

struct MLP: Layer {
    var layer1 = Dense<Float>(inputSize: 2, outputSize: 10, activation: relu)
    var layer2 = Dense<Float>(inputSize: 10, outputSize: 1, activation: relu)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        let h0 = layer1(input).withDerivative { print("∂L/∂layer1 =", $0) }
        return layer2(h0)
    }
}

var classifier = MLP()
let optimizer = SGD(for: classifier, learningRate: 0.02)

let x: Tensor<Float> = [[0, 0], [0, 1], [1, 0], [1, 1]]
let y: Tensor<Float> = [0, 1, 1, 0]

for _ in 0..<10 {
    let 𝛁model = gradient(at: classifier) { classifier -> Tensor<Float> in
        let ŷ = classifier(x).withDerivative { print("∂L/∂ŷ =", $0) }
        let loss = (ŷ - y).squared().mean()
        print("Loss: \(loss)")
        return loss
    }
    optimizer.update(&classifier, along: 𝛁model)
}

## バックプロパゲーション中の活性化を再計算してメモリを節約する（チェックポインティング）

チェックポインティングは、メモリを節約するためにリバースモード自動微分をする従来の手法です。元の計算で大きな中間値を保存して導関数計算をするのではなく、必要に応じてバックプロパゲーション中に中間値を再計算します。

この手法は、最近のディープラーニングライブラリでも実現されています。Swift では、[`withRecomputationInPullbacks(_:)`](https://www.tensorflow.org/swift/api_docs/Protocols/Differentiable#/s:10TensorFlow14DifferentiablePAAE28withRecomputationInPullbacksyqd__qd__xcAaBRd__lF) API によってバックプロパゲーション中に何を再計算するかを制御することができ、すべての `Differentiable` 型で利用可能です。

しかしここでは、数行のコードのみを使用して独自の勾配チェックポインティング API を新規に定義する方法を見ていきます。

### 勾配チェックポインティング API

独自の勾配チェックポインティング API である `makeRecomputedInGradient(_:)` を定義することができますが、標準ライブラリ関数 [`differentiableFunction(from:)`](https://www.tensorflow.org/swift/api_docs/Functions#/s:10TensorFlow22differentiableFunction4fromq0_x_q_tcq0_5value_15CotangentVectorQz_AEQy_tAEQy0_c8pullbacktx_q_tc_tAA14DifferentiableRzAaJR_AaJR0_r1_lF) の観点から見ると、これは微分関数（「ベクトルヤコビアン積（VJP）関数」とも呼ばれる）から微分可能な関数を直接作成するための省略表現です。

これまで説明したように、微分関数は元の関数の結果とプルバッククロージャのタプルを返します。`original(x)` を `value:` で返し、`original` 上で`pullback(at:in:)` を呼び出すことにより、元の関数を再評価してプルバックを取得します。

In [ ]:
/// Given a differentiable function, returns the same differentiable function except when
/// derivatives of this function are being computed. In that case, values in the original function needed
/// for computing the derivatives will be recomputed, instead of being captured by the differential or pullback.
///
/// - Parameter body: The body of the differentiable function.
/// - Returns: The same differentiable function whose derivatives, when computed, will recompute
///   some values from the original function.
func makeRecomputedInGradient<T: Differentiable, U: Differentiable>(
    _ original: @escaping @differentiable (T) -> U
) -> @differentiable (T) -> U {
    return differentiableFunction { x in
        (value: original(x), pullback: { v in pullback(at: x, in: original)(v) })
    }
}

### 動作を確認する

In [ ]:
let input: Float = 10.0
print("Running original computation...")

// Differentiable multiplication with checkpointing.
let square = makeRecomputedInGradient { (x: Float) -> Float in
    print("  Computing square...")
    return x * x
}

// Differentiate `f(x) = (cos(x))^2`.
let (output, backprop) = valueWithPullback(at: input) { input -> Float in
    return square(cos(input))
}
print("Running backpropagation...")
let grad = backprop(1)
print("Gradient = \(grad)")

### ニューラルネットワークモジュールに拡張する

この例では、単純な畳み込みニューラルネットワークを定義します。

```swift
struct Model: Layer {     var conv = Conv2D<Float>(filterShape: (5, 5, 3, 6))     var maxPool = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))     var flatten = Flatten<Float>()     var dense = Dense<Float>(inputSize: 36 * 6, outputSize: 10)      @differentiable     func call(_ input: Tensor<Float>) -> Tensor<Float> {         return input.sequenced(through: conv, maxPool, flatten, dense)     } }
```

バックプロパゲーション中に畳み込みレイヤー（`conv`）の活性化を再計算させる必要があります。しかし、`makeRecomputedInGradient(_:)` を使用すると、特に [`sequenced(in:through:_:_:_:_:)`](https://www.tensorflow.org/swift/api_docs/Protocols/Differentiable#/s:10TensorFlow14DifferentiablePAAE9sequenced2in7through____6OutputQyd_3_AA7ContextC_qd__qd_0_qd_1_qd_2_qd_3_t5InputQyd__RszAA5LayerRd__AaMRd_0_AaMRd_1_AaMRd_2_AaMRd_3_AKQyd_0_AGRtd__AKQyd_1_AGRtd_0_AKQyd_2_AGRtd_1_AKQyd_3_AGRtd_2_r3_lF) を用いて順次レイヤーを適用する場合は、結果のコードが扱いにくく見える可能性があります。

```swift
input.sequenced(in: context, through: conv, maxPool, flatten, dense)
```

そこで、バックプロパゲーション中にレイヤーをラップして活性化を再計算させる、**特殊レイヤー型**を定義してみてはどうでしょう。早速、これを試してみます。

まず、二値関数を取る `makeRecomputedInGradient(_:)` 関数を定義します。

In [ ]:
// Same as the previous `makeRecomputedInGradient(_:)`, except it's for binary functions.
func makeRecomputedInGradient<T: Differentiable, U: Differentiable, V: Differentiable>(
    _ original: @escaping @differentiable (T, U) -> V
) -> @differentiable (T, U) -> V {
    return differentiableFunction { x, y in
        (value: original(x, y), pullback: { v in pullback(at: x, y, in: original)(v) })
    }
}

次に、汎用レイヤー `ActivationDiscarding<Wrapped>` を定義します。

In [ ]:
import TensorFlow

/// A layer wrapper that makes the underlying layer's activations be discarded during application
/// and recomputed during backpropagation.
struct ActivationDiscarding<Wrapped: Layer>: Layer {
    /// The wrapped layer.
    var wrapped: Wrapped

    @differentiable
    func callAsFunction(_ input: Wrapped.Input) -> Wrapped.Output {
        let apply = makeRecomputedInGradient { (layer: Wrapped, input: Input) -> Wrapped.Output in
            print("    Applying \(Wrapped.self) layer...")
            return layer(input)
        }
        return apply(wrapped, input)
    }
}

最後に、アプリケーション中に活性化が破棄されバックプロパゲーション中に再計算されることを除き、同じレイヤーを返すメソッドを全てのレイヤーに追加することができます。

In [ ]:
extension Layer {
    func discardingActivations() -> ActivationDiscarding<Self> {
        return ActivationDiscarding(wrapped: self)
    }
}

モデルに関しては、ここで変更が必要なのは畳み込みレイヤーを活性化再計算レイヤーにラップすることのみです。

```swift
var conv = Conv2D<Float>(filterShape: (5, 5, 3, 6)).discardingActivations()
```

あとは、それをモデルで使用するだけです！

In [ ]:
struct Model: Layer {
    var conv = Conv2D<Float>(filterShape: (5, 5, 3, 6)).discardingActivations()
    var maxPool = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))
    var flatten = Flatten<Float>()
    var dense = Dense<Float>(inputSize: 36 * 6, outputSize: 10)

    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: conv, maxPool, flatten, dense)
    }
}

トレーニングループを実行すると、畳み込みレイヤーの活性化が 2 回（レイヤー適用中に 1 回、バックプロパゲーション中に 1 回）計算されていることが分かります。

In [ ]:
// Use random training data.
let x = Tensor<Float>(randomNormal: [10, 16, 16, 3])
let y = Tensor<Int32>(rangeFrom: 0, to: 10, stride: 1)

var model = Model()
let opt = SGD(for: model)

for i in 1...5 {
    print("Starting training step \(i)")
    print("  Running original computation...")
    let (logits, backprop) = model.appliedForBackpropagation(to: x)
    let (loss, dL_dŷ) = valueWithGradient(at: logits) { logits in
        softmaxCrossEntropy(logits: logits, labels: y)
    }
    print("  Loss: \(loss)")
    print("  Running backpropagation...")
    let (dL_dθ, _) = backprop(dL_dŷ)
    
    opt.update(&model, along: dL_dθ)
}

このように、一般的な可微分プログラミングライブラリを簡単に異なるドメインに定義できます。